<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/restructure_code/StarpilotProject_Implementing_evalutation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [1]:
!pip install procgen

     |████████████████████████████████| 39.9MB 84kB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
     |████████████████████████████████| 204kB 60.6MB/s 
     |████████████████████████████████| 22.2MB 1.2MB/s 
     |████████████████████████████████| 665kB 57.5MB/s 
     |████████████████████████████████| 3.3MB 54.7MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [2]:
#Clone git
!git clone -b restructure_code https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

Cloning into 'my_project'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 153 (delta 87), reused 19 (delta 0), pack-reused 0
Receiving objects: 100% (153/153), 772.70 KiB | 12.88 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [3]:
#update git
%cd /content/my_project
! git pull

/content/my_project
Already up to date.


In [4]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Mounted at /content/drive


# TESTING IMPLEMENTATION

In [5]:
from baseline import set_hyperparameters
import hyperparameters as h
from ppo import PPO
import torch

In [6]:
set_hyperparameters(baseline='Procgen')
model = PPO(print_output=True)

Observation space: Box(0.0, 1.0, (3, 64, 64), float32)
Action space: 15


In [ ]:
policy = model.train()

Step: 8192	Mean reward: 3.8125
Step: 16384	Mean reward: 4.21875
Step: 24576	Mean reward: 4.375
Step: 32768	Mean reward: 4.46875
Step: 40960	Mean reward: 5.0
Step: 49152	Mean reward: 4.3125
Step: 57344	Mean reward: 5.46875
Step: 65536	Mean reward: 5.28125
Step: 73728	Mean reward: 4.9375
Step: 81920	Mean reward: 6.6875
Step: 90112	Mean reward: 6.1875
Step: 98304	Mean reward: 6.8125
Step: 106496	Mean reward: 7.625
Step: 114688	Mean reward: 8.03125
Step: 122880	Mean reward: 8.15625
Step: 131072	Mean reward: 8.75
Step: 139264	Mean reward: 9.1875
Step: 147456	Mean reward: 8.28125
Step: 155648	Mean reward: 9.1875
Step: 163840	Mean reward: 8.5625
Step: 172032	Mean reward: 10.375
Step: 180224	Mean reward: 9.3125
Step: 188416	Mean reward: 11.09375
Step: 196608	Mean reward: 10.3125
Step: 204800	Mean reward: 9.75
Step: 212992	Mean reward: 8.40625
Step: 221184	Mean reward: 10.1875
Step: 229376	Mean reward: 10.0625
Step: 237568	Mean reward: 10.75
Step: 245760	Mean reward: 9.59375
Step: 253952	Mean r

KeyboardInterrupt: ignored

In [ ]:
tools.create_index_table_from_txt_files()

,file_name,total_steps,num_envs,num_levels,num_steps,num_epochs,batch_size,eps,grad_eps,value_coef,entropy_coef,lr,opt_extra,gamma,lmbda,version,optimizer,Time limit,Time spent (in seconds),Steps taken,Done,time_limit,Time spent (in seconds),Steps taken,Done,Last Mean Reward
Value,Baseline(PPO)_Run_15Nov_13h11m00s,8000000.0,32,10,128,3,256,0.1,0.5,1,0.01,0.00025,1e-05,0.99,0.95,Baseline(PPO),Adam,12:30:0,11253.99,8003584,True,NaN,NaN,NaN,NaN,10.0625
Value,Baseline(Procgen)_Run_15Nov_16h18m34s,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0005,1e-05,0.999,0.95,Baseline(Procgen),Adam,12:30:0,10962.17,8003584,True,NaN,NaN,NaN,NaN,19.71875
Value,Baseline(PPO)_Run_16Nov_20h08m12s,8000000.0,32,10,128,3,256,0.1,0.5,1,0.01,0.00025,1e-05,0.99,0.95,Baseline(PPO),Adam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0625
Value,Baseline(Procgen)_Run_17Nov_14h07m08s,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0005,NaN,0.999,0.95,Baseline(Procgen),NaN,NaN,NaN,NaN,NaN,43200,16572.48,8003584,False,20.1875
Value,Baseline(Procgen)_Run_21Nov_12h43m19s,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0005,NaN,0.999,0.95,Baseline(Procgen),NaN,NaN,NaN,NaN,NaN,43200,NaN,NaN,NaN,6.375


# Testing evaluation

In [18]:
from model import Encoder
from policy import Policy
import utils
import hyperparameters as h
import torch.nn as nn
import imageio

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

feature_dim= 512    # <- The only thing we chose ourself

#Fixed values
in_channels = 3 #RGB
num_actions = 15  #Number of actions in the Procgen environment


MODEL_PATH = '/content/drive/My Drive/02456-Deep-Learning-Project/Models/'

def load_policy(file_name):
  encoder = Encoder(in_channels = in_channels, feature_dim = feature_dim)  
  
  policy = Policy(encoder = encoder, feature_dim = feature_dim, num_actions = num_actions)
  policy.cuda()
  policy.load_state_dict(torch.load(MODEL_PATH + file_name + '.pt')["policy_state_dict"])
  policy.cuda() 

  return policy

In [20]:
def policy_evaluation(video_name='vid', print_output=False, policy=None, file_name=None):
  if policy is None and file_name is None:
    raise ValueError("Policy should be specified by file_name or given as object")
  if policy is None:
    policy = load_policy(file_name)
  
  # Make evaluation environment
  eval_env = utils.make_env(h.num_envs, start_level=h.num_levels, num_levels=h.num_levels)
  obs = eval_env.reset()

  frames = []
  total_reward = []
  
  # Evaluate policy
  policy.eval()
  for _ in range(512):  #<--- Hardcoded batch size?

    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)

  # Calculate average return
  total_reward = torch.stack(total_reward).sum(0).mean(0)
  
  if print_output:
    print('Average return:', total_reward)

  # Save frames as video
  frames = torch.stack(frames).cpu().numpy()
  imageio.mimsave(video_name+'.mp4', frames, fps=25)
  
  return total_reward

In [ ]:
#set_hyperparameters(baseline='Procgen')

INDEX_TABLE = tools.create_index_table_from_txt_files()

#file_name = INDEX_TABLE['file_name'][3].strip()
#print(file_name)
#loaded_policy = load_policy(file_name=file_name)
policy_evaluation(print_output=True, policy = policy)
#policy_evaluation(print_output=True, file_name=file_name)

NameError: ignored

In [ ]:
#INDEX_TABLE
loaded_policy.act(utils.make_env(h.num_envs, start_level=h.num_levels, num_levels=h.num_levels).reset())

(tensor([ 7,  8,  0, 14, 13, 12,  8,  9,  3, 12,  0,  1,  7, 14,  4,  3, 13,  6,
          7, 11, 12,  2, 12,  6,  2,  7,  2,  6, 14, 11, 11,  9]),
 tensor([-2.9727, -2.0689, -3.4070, -2.2008, -1.5579, -2.2801, -2.0689, -1.1039,
         -1.8192, -2.8460, -2.0766, -2.7992, -2.6830, -3.1655, -2.5775, -1.8192,
         -1.8571, -2.0758, -2.6830, -1.0149, -2.2021, -1.1738, -2.0179, -2.0758,
         -1.1738, -3.3540, -3.8563, -2.6099, -2.5835, -2.0599, -0.7330, -1.1039]),
 tensor([0.0300, 0.0371, 0.0257, 0.0371, 0.0300, 0.0202, 0.0371, 0.0296, 0.0300,
         0.0296, 0.0313, 0.0257, 0.0371, 0.0217, 0.0223, 0.0300, 0.0313, 0.0296,
         0.0371, 0.0231, 0.0257, 0.0217, 0.0223, 0.0296, 0.0217, 0.0217, 0.0313,
         0.0202, 0.0223, 0.0257, 0.0144, 0.0296]))

In [ ]:
def give_me_numbers():
  return 2.0, 3.0

def add_to_data_file(string, file_name):
    f = open(DATA_PATH + file_name, "a")
    f.write(string)
    f.close()
  
test_text = ''
for x in give_me_numbers():
  test_text += str(x)
  test_text += ','
add_to_data_file(test_text,"test.csv")

In [24]:
from model import Encoder
from policy import Policy
import utils
import hyperparameters as h
import torch.nn as nn
import imageio

INDEX_TABLE = tools.create_index_table_from_txt_files()

file_name = INDEX_TABLE['file_name'][-2].strip()
print(file_name)
checkpoint = torch.load(MODEL_PATH + file_name + '.pt')
checkpoint["optimizer_state_dict"]

feature_dim= 512    # <- The only thing we chose ourself

#Fixed values
in_channels = 3 #RGB
num_actions = 15  #Number of actions in the Procgen environment

encoder = Encoder(in_channels = in_channels, feature_dim = feature_dim)  
policy = Policy(encoder = encoder, feature_dim = feature_dim, num_actions = num_actions)
policy.cuda()
policy.load_state_dict(torch.load(MODEL_PATH + file_name + '.pt')["policy_state_dict"])
policy.cuda() 

policy_evaluation(print_output=True, policy = policy)

Baseline(Procgen)_Run_22Nov_14h18m57s
Average return: tensor(14.2990)


tensor(14.2990)

In [10]:
INDEX_TABLE.iloc[-2]

file_name                    Baseline(Procgen)_Run_22Nov_14h18m57s
total_steps                                              8000000.0
num_envs                                                        32
num_levels                                                      10
num_steps                                                      256
num_epochs                                                       3
batch_size                                                     512
eps                                                            0.2
grad_eps                                                       0.5
value_coef                                                     0.5
entropy_coef                                                  0.01
lr                                                          0.0005
opt_extra                                                      NaN
gamma                                                        0.999
lmbda                                                         

In [23]:
#torch.load(MODEL_PATH + file_name + '.pt')["policy_state_dict"]
#torch.load(MODEL_PATH + file_name + '.pt')

{'optimizer_state_dict': {'param_groups': [{'amsgrad': False,
    'betas': (0.9, 0.999),
    'eps': 1e-05,
    'lr': 0.0005,
    'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'weight_decay': 0}],
  'state': {0: {'exp_avg': tensor([[[[-2.8541e-05, -2.3944e-05,  8.7455e-05,  ..., -1.9149e-04,
               -5.5427e-05, -2.3906e-04],
              [-3.4036e-05,  1.1501e-04,  2.0186e-04,  ..., -3.1350e-04,
               -2.4918e-04, -2.5993e-04],
              [ 2.3736e-04, -5.1093e-05, -1.7345e-04,  ..., -3.3710e-04,
                2.3059e-04,  1.8850e-04],
              ...,
              [ 1.9734e-04,  1.6508e-04, -2.8427e-05,  ..., -1.2638e-04,
               -1.7117e-04,  1.6079e-05],
              [ 7.0818e-05,  1.0917e-04,  1.3113e-04,  ..., -1.8879e-05,
               -1.1677e-04, -1.5269e-04],
              [ 1.6662e-04,  1.8056e-04,  7.4023e-05,  ..., -1.6827e-04,
                8.1543e-05, -1.0887e-05]],
    
             [[-3.2873e-05, -5.9333e-05,  6.9758e-05,  ...